In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
num_epochs = 10
batch_size = 32
learning_rate = 0.001

In [3]:
class ModifiedEfficientNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedEfficientNet, self).__init__()
        # Load pretrained EfficientNetB0
        efficientnet = torchvision.models.efficientnet_b0(pretrained=True)
        
        # Keep all layers except the classifier
        self.features = efficientnet.features
        
        # Freeze the feature layers
        for param in self.features.parameters():
            param.requires_grad = False
            
        # Create new classifier layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(1280, 256),  # EfficientNetB0 outputs 1280 features
            nn.ReLU(True),
            nn.Linear(256, num_classes)
        )
                
    def forward(self, x):
        x = self.features(x)
        x = x.mean([2, 3])  # Global average pooling
        x = self.classifier(x)
        return x


In [4]:
# Load and preprocess the SVHN dataset
transform = transforms.Compose([
    transforms.Resize(224),  # EfficientNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.SVHN(
    root='./data', 
    split='train',
    download=True, 
    transform=transform
)

test_dataset = torchvision.datasets.SVHN(
    root='./data', 
    split='test',
    download=True, 
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)


100%|███████████████████████████████████████████████████████████████| 182040794/182040794 [02:38<00:00, 1151403.58it/s]


100%|█████████████████████████████████████████████████████████████████| 64275384/64275384 [00:58<00:00, 1092610.90it/s]


In [5]:
# Initialize the model
model = ModifiedEfficientNet(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)


C:\Users\Besose\anaconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Besose\anaconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Besose/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|█████████████████████████████████████████████████████████████████████████████| 20.5M/20.5M [00:21<00:00, 1.02MB/s]


In [6]:
# Training function
def train_model():
    print("Starting training...")
    model.train()
    total_step = len(train_loader)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()
            
            if (i + 1) % 50 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], '
                      f'Loss: {loss.item():.4f}, '
                      f'Accuracy: {100 * correct / total:.2f}%')
        
        print(f'Epoch [{epoch+1}/{num_epochs}] completed. '
              f'Average Loss: {running_loss/total_step:.4f}, '
              f'Final Accuracy: {100 * correct / total:.2f}%')

In [7]:
# Evaluation function
def evaluate_model():
    print("\nEvaluating model on test set...")
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f'Test Accuracy: {100 * correct / total:.2f}%')

In [8]:
train_model()

Starting training...
Epoch [1/10], Step [50/2290], Loss: 2.1807, Accuracy: 20.69%
Epoch [1/10], Step [100/2290], Loss: 1.9471, Accuracy: 24.75%
Epoch [1/10], Step [150/2290], Loss: 2.0606, Accuracy: 27.17%
Epoch [1/10], Step [200/2290], Loss: 1.8680, Accuracy: 28.45%
Epoch [1/10], Step [250/2290], Loss: 1.9592, Accuracy: 29.73%
Epoch [1/10], Step [300/2290], Loss: 1.8616, Accuracy: 30.64%
Epoch [1/10], Step [350/2290], Loss: 1.7266, Accuracy: 31.43%
Epoch [1/10], Step [400/2290], Loss: 1.7373, Accuracy: 32.05%
Epoch [1/10], Step [450/2290], Loss: 1.7073, Accuracy: 32.90%
Epoch [1/10], Step [500/2290], Loss: 1.7073, Accuracy: 33.59%
Epoch [1/10], Step [550/2290], Loss: 1.6128, Accuracy: 34.18%
Epoch [1/10], Step [600/2290], Loss: 1.7264, Accuracy: 34.45%
Epoch [1/10], Step [650/2290], Loss: 1.6300, Accuracy: 34.88%
Epoch [1/10], Step [700/2290], Loss: 1.7877, Accuracy: 35.28%
Epoch [1/10], Step [750/2290], Loss: 1.7426, Accuracy: 35.45%
Epoch [1/10], Step [800/2290], Loss: 1.8578, Accur

In [9]:
evaluate_model()


Evaluating model on test set...
Test Accuracy: 53.28%
